## 1. Multipage Tabular Scrape

- <a href="https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AllRecordsAction.action">On this site</a>, scrape all doctors whose last names begin with "Z".
- Export the content into a CSV file called ```md_Z.csv```.


In [ ]:
### add more cells as needed

In [1]:
from bs4 import BeautifulSoup  ## web scraping
import requests ## request html for a page(s)
import pandas as pd ## pandas to work with data

#### Uhhhhh, I hope this isn't cheating, but it looks like you 
## can just go to the list that contains all the Z doctors sooo:

In [5]:
url = \
"https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AlphabetSearchAction.action?alpbhabetSearch=Z&d-49653-p=1"
response = requests.get(url)
response.status_code

200

In [4]:
type(response.text)

str

In [ ]:
## Oh dang, that's page one. It looks like there's more. 
## I need the urls for all the pages
##can we make a function? 

In [6]:
base_url = "https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AlphabetSearchAction.action?alpbhabetSearch=Z&d-49653-p={}"
def find_url(page):
    full_url = base_url.format(page)
    return(full_url)
    

In [7]:
## Now I guess put the function in a forloop?

all_urls = []
for url in range (1,6):
    all_urls.append(find_url(url))

In [8]:
## Did it capture all of them? 
all_urls

['https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AlphabetSearchAction.action?alpbhabetSearch=Z&d-49653-p=1',
 'https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AlphabetSearchAction.action?alpbhabetSearch=Z&d-49653-p=2',
 'https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AlphabetSearchAction.action?alpbhabetSearch=Z&d-49653-p=3',
 'https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AlphabetSearchAction.action?alpbhabetSearch=Z&d-49653-p=4',
 'https://apps.health.ny.gov/pubdoh/professionals/doctors/conduct/factions/AlphabetSearchAction.action?alpbhabetSearch=Z&d-49653-p=5']

In [9]:
## I don't think we need BeautifulSoup because it's a table? 
from random import randrange
import time
#Lets try to use functions in our scrape? 


In [46]:
def scrape(list_name):
    counter = 1
    all_df = []
    busted_links = []
    for link in list_name:
        print(f"Scraping {counter} of {len(list_name)}")
        counter += 1
        response = requests.get(link)
        if response.status_code == 200:
            df = pd.read_html(response.text)
            all_df.append(df[0])
        else:
            print(f"{link} returned busted link with response {response.status_code}")
            busted_links.append(link) 
    snoozer = randrange(5, 12)
    print(f"Snoozing for {snoozer} seconds before next scrape")
    time.sleep(snoozer)
    return all_df
    return busted_links

    ##print(link)

In [53]:
## lets try this out
docs = scrape(all_urls)

Scraping 1 of 5
Scraping 2 of 5
Scraping 3 of 5
Scraping 4 of 5
Scraping 5 of 5
Snoozing for 7 seconds before next scrape


In [58]:
type(docs)

list

In [54]:
## I don't understand why it looks like that
docs


[                                  Physician Last Name  \
 0                                             Zaccheo   
 1                                           Zachariah   
 2                                              Zachel   
 3                                              Zackin   
 4                                              Zackin   
 5                                              Zackin   
 6                                               Zadeh   
 7                                               Zafar   
 8                                               Zafar   
 9                                                Zahl   
 10                                             Zahler   
 11                                              Zaino   
 12                                                Zak   
 13                                               Zaki   
 14                                              Zales   
 15                                           Zalmanov   
 16           

In [51]:
##That seems like a lot of stuff... I"m not sure if it worked
## Maybe I'll be able to tell when I put it together
## I feel like this would have worked better if I made several simple functions
## and included them as commands within in the forloop 

In [52]:
##we can for sure make a concat function

def concat(list):
    df = pd.concat(list, ignore_index = True)
    return df

In [55]:
z_docs = concat(docs)

In [56]:
z_docs

,Physician Last Name,Physician First Name,Physician Middle Name,License Number,License Type,Effective Date,Date Updated,Year of Birth
0,Zaccheo,Jerald,D,134842.0,MD,12/20/2001,12/23/2001,1946.0
1,Zachariah,Abraham,NaN,137458.0,MD,09/15/2004,09/08/2004,1950.0
2,Zachel,Gretchen,NaN,20699.0,PA,10/13/2017,10/06/2017,1952.0
3,Zackin,Henry,J,101457.0,MD,03/28/2002,03/09/2005,1941.0
4,Zackin,Henry,J,101457.0,MD,03/16/2005,03/09/2005,1941.0
...,...,...,...,...,...,...,...,...
89,Zuniga,Dario,NaN,123324.0,MD,05/07/2002,05/07/2002,1941.0
90,Zuttah,Silas,H,153216.0,MD,01/22/2003,06/17/2003,1953.0
91,Zweig,Steven,Jeffrey,140242.0,MD,05/17/2006,05/10/2006,1949.0
92,New Physician Search,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
##WTF there's weird random stuff going on
## for some reason the last two rows of each list are not names. 
## can I splice those rows off? maybe with lc method

z_docs_clean = []
for df in docs:
    for row in df:
        clean = df[0:-2]
        z_docs_clean.append(clean)
        
z_docs_clean

[   Physician Last Name Physician First Name Physician Middle Name  \
 0              Zaccheo               Jerald                     D   
 1            Zachariah              Abraham                   NaN   
 2               Zachel             Gretchen                   NaN   
 3               Zackin                Henry                     J   
 4               Zackin                Henry                     J   
 5               Zackin                Henry                     J   
 6                Zadeh               Mehran                   NaN   
 7                Zafar                Kamal                   NaN   
 8                Zafar                Syeda                   NaN   
 9                 Zahl              Kenneth                   NaN   
 10              Zahler               Gideon                   NaN   
 11               Zaino               Edward                   NaN   
 12                 Zak                 John                   NaN   
 13                Z

In [62]:
##oh good got rid of them
## now to concat the clean data
## I forgot that we did combine and export all in one step last week

def combineDFS(list_name, filename):
    df = pd.concat(list_name, ignore_index = True)
    df.to_csv(filename, encoding = "UTF-8", index = False)
    print(f"{filename} is in your current folder")
    return(df)


In [63]:
md_z_df = combineDFS(z_docs_clean, "md_z.csv" )

md_z.csv is in your current folder


# 2. Conversion function


Write a function that takes string values like ```$12.24267```, ```10,201``` and ```$12,501``` and converts them into floating point numbers like ```12.24```, ```10201.0``` and ```12501.0```

Test it out on those 3 string values.




In [77]:
def conversion(currency):
    new_float = []
    for number in currency:
        new_float.append(float(number.replace(',', '').replace('$', '')))
    return new_float

In [65]:
money = ["$12.24267", "10,201", "$12,501"]

In [78]:
str_to_num = conversion(money)
str_to_num

[12.24267, 10201.0, 12501.0]

In [8]:
### add more cells as needed